# GOLD Tables

In this notebook we will explore the possible analytics that can be extracted from the silver layer.

## 1. Claims Analysis
 Calculate total and average claim amounts and claim counts grouped by region, car use, age group, and credit score band.

In [1]:
from policyML.bronze.bronze import get_db_connection
import pandas as pd

In [14]:
conn = get_db_connection()
schema = 'silver'
table_name = 'insurance'

query = f'SELECT * FROM "{schema}"."{table_name}";'
df = pd.read_sql(query, conn)
conn.close()

df.info()

/var/folders/lx/thnwywfs79x9xsl2bq8yrjy00000gn/T/ipykernel_3017/3495310533.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       25000 non-null  int64         
 1   insurance_coverage_days  25000 non-null  int64         
 2   insured_age              25000 non-null  int64         
 3   insured_sex              25000 non-null  int64         
 4   insured_status           25000 non-null  int64         
 5   insured_noclaimyears     25000 non-null  int64         
 6   insured_creditscore      25000 non-null  int64         
 7   insured_regionid         25000 non-null  int64         
 8   insured_drivearea        25000 non-null  int64         
 9   car_use                  25000 non-null  int64         
 10  car_age                  25000 non-null  int64         
 11  annualdrive_km           25000 non-null  int64         
 12  drive_avgdays_week       25000 n

### Claim by Region

In [ ]:
bins_age = [0,25,35,45,55,65,120]
labels_age = ['<25','25-34','35-44','45-54','55-64','65+']

df['agegroup'] = pd.cut(df['insured_age'], bins=bins_age, labels=labels_age, right=False)

bins_credit = [0,600,700,800,900,1000]
labels_credit = ['<600','600-699','700-799','800-899','>=900']

df['creditscoreband'] = pd.cut(df['insured_creditscore'], bins=bins_credit, labels=labels_credit, right=False)

# Total and average claim amounts and counts by region
claims_by_region = df.groupby('insured_regionid').agg(
    total_claim_amount=('amt_claim','sum'),
    avg_claim_amount=('amt_claim','mean'),
    total_claims=('nb_claim','sum'),
    avg_claims=('nb_claim','mean'),
    policy_count=('id','count')
)
claims_by_region.sort_values('total_claims', ascending=False).head()

,total_claim_amount,avg_claim_amount,total_claims,avg_claims,policy_count
insured_regionid,,,,,
43,178306.16,175.844339,39,0.038462,1014
39,93102.19,105.200215,37,0.041808,885
35,140020.27,184.479934,37,0.048748,759
30,165284.13,231.166615,35,0.048951,715
32,134453.68,231.020069,31,0.053265,582


### Claim by car use

In [ ]:
claims_by_use = df.groupby('car_use').agg(
    total_claim_amount=('amt_claim','sum'),
    avg_claim_amount=('amt_claim','mean'),
    total_claims=('nb_claim','sum'),
    avg_claims=('nb_claim','mean'),
    policy_count=('id','count')
)
claims_by_use

,total_claim_amount,avg_claim_amount,total_claims,avg_claims,policy_count
car_use,,,,,
0,258805.35,395.726835,56,0.085627,654
1,4153072.48,331.635589,712,0.056855,12523
2,14618.59,40.160962,5,0.013736,364
3,1489455.47,129.981278,405,0.035343,11459


### Claim by age group

In [34]:
 # Total and average claim amounts and counts by age group
claims_by_age = df.groupby('agegroup').agg(
    total_claim_amount=('amt_claim','sum'),
    avg_claim_amount=('amt_claim','mean'),
    total_claims=('nb_claim','sum'),
    avg_claims=('nb_claim','mean'),
    policy_count=('id','count')
)
claims_by_age

/var/folders/lx/thnwywfs79x9xsl2bq8yrjy00000gn/T/ipykernel_3017/1751958123.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  claims_by_age = df.groupby('agegroup').agg(


,total_claim_amount,avg_claim_amount,total_claims,avg_claims,policy_count
agegroup,,,,,
<25,267691.67,395.993595,46,0.068047,676
25-34,1547116.47,420.755091,221,0.060103,3677
35-44,1056394.79,246.016486,225,0.052399,4294
45-54,1665128.99,307.503045,309,0.057064,5415
55-64,884666.29,169.346533,238,0.045559,5224
65+,494953.68,86.621225,139,0.024326,5714


## 2. Driving Behavior Segmentation

### Claim by age and hard events

In [ ]:
# Define hard events as sums of high-deceleration/acceleration counts
df['hard_brakes'] = df['brake_06ms2_1000km'] + df['brake_07ms2_1000km']
df['hard_accelerations'] = df['accel_06ms2_1000km'] + df['accel_07ms2_1000km']

# Average hard events by driver age group
behavior_by_age = df.groupby('agegroup')[['hard_brakes','hard_accelerations']].mean()
behavior_by_age


/var/folders/lx/thnwywfs79x9xsl2bq8yrjy00000gn/T/ipykernel_3017/263073144.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  behavior_by_age = df.groupby('agegroup')[['hard_brakes','hard_accelerations']].mean()


,hard_brakes,hard_accelerations
agegroup,,
<25,1.251479,1.130178
25-34,1.119663,0.994289
35-44,0.561015,0.573125
45-54,0.626223,0.577655
55-64,0.387825,0.289051
65+,0.702660,0.608155


### Average hard events by car age group

In [ ]:
bins_car_age = [0,6,11,16,21]
labels_car_age = ['0-5','6-10','11-15','16-20']
df['CarAgeGroup'] = pd.cut(df['car_age'], bins=bins_car_age, labels=labels_car_age, right=False)
behavior_by_car = df.groupby('CarAgeGroup')[['hard_brakes','hard_accelerations']].mean()
behavior_by_car

/var/folders/lx/thnwywfs79x9xsl2bq8yrjy00000gn/T/ipykernel_3017/1978256443.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  behavior_by_car = df.groupby('CarAgeGroup')[['hard_brakes','hard_accelerations']].mean()


,hard_brakes,hard_accelerations
CarAgeGroup,,
0-5,0.323802,0.254836
6-10,0.835909,0.784240
11-15,1.657287,1.512488
16-20,0.857651,0.814947


## 3. Risk Profiling

- Create a risk score for each driver combining number of claims, aggressive maneuvers, and credit score (higher score means lower risk).
- Classify drivers into High vs Low risk based on the median score.
- Summarize counts of high/low risk drivers by marital status, sex, and region.


In [ ]:

import numpy as np
# Simple risk score: more claims or harsh events increase risk, higher credit reduces risk
df['RiskScore'] = df['nb_claim'] * 5 + (df['aggressive'])/10 + (900 - df['insured_creditscore'])/50
threshold = df['RiskScore'].median()
df['RiskCategory'] = np.where(df['RiskScore'] >= threshold, 'High', 'Low')

# Distribution of risk categories
risk_by_status = df.groupby(['insured_status','RiskCategory']).size().unstack(fill_value=0)
risk_by_sex = df.groupby(['insured_sex','RiskCategory']).size().unstack(fill_value=0)
risk_by_region = df.groupby(['insured_regionid','RiskCategory']).size().unstack(fill_value=0)

print(risk_by_status)
print(risk_by_sex)
print(risk_by_region.head())

RiskCategory    High   Low
insured_status            
0               4533  2986
1               8087  9394
RiskCategory  High   Low
insured_sex             
0             6785  6666
1             5835  5714
RiskCategory      High  Low
insured_regionid           
11                   3   10
12                 150  167
13                 162  173
14                 164  176
15                 250  216


## 4. Usage Pattern Clustering (NOT IMPLEMENTED)

- Segment drivers based on average driving days per week.

In [32]:
# Segment drivers by average driving days/week
bins_days = [0,2,4,7]
labels_days = ['Low (<=2)','Medium (2-4)','High (4-7))']

df['UsageSegment'] = pd.cut(df['drive_avgdays_week'], bins=bins_days, labels=labels_days, right=False)
usage_distribution = df['UsageSegment'].value_counts().sort_index()
usage_distribution

UsageSegment
Low (<=2)       2
Medium (2-4)    2
High (4-7))     5
Name: count, dtype: int64